Potential Notebook for specific synapse setups, but might be crasahing server

In [ ]:
import sys
sys.path.append("../")
sys.path.append("../Modules/")

from Modules.simulation import Simulation
from Modules.cell_builder import SkeletonCell, CellBuilder
from Modules.constants import HayParameters

import os
from neuron import h

from logger import Logger # type: ignore

import numpy as np

In [ ]:
%cd ../scripts

In [ ]:
try:
    os.system("rm -r x86_64/")
except:
    pass

In [ ]:
# initialize simulation object and parameters
sim = Simulation(SkeletonCell.Hay)
parameters = HayParameters(sim_name = 'testingSynapticSum', h_tstop= 1000, all_synapses_off=True) # NOTE: synapses will crash the simulation since they are not mapped and will remain on deleted sections

In [ ]:
os.system(f"nrnivmodl {sim.cell_type.value['modfiles']} > /dev/null 2>&1")

h.load_file('stdrun.hoc')
h.nrn_load_dll('./x86_64/.libs/libnrnmech.so')

In [ ]:
logger = Logger(None)
cell_builder = CellBuilder(sim.cell_type, parameters, logger)

In [ ]:
complex_cell, _ = cell_builder.build_cell()

In [ ]:
# def get_first_dendrite(root_sec):
#     # get list of segments along one terminal path from this root
#     dendrite_segs = []
#     sec_children = root_sec.children()
#     while len(sec_children) > 0:
#         for sec in sec_children:
#             dendrite_segs.extend(list(sec))
#         new_sec = sec_children[0]
#         sec_children = new_sec.children()
#     return dendrite_segs

# # dendrite path chosen is just the first basal root sec and the first of its children
# basal_root = complex_cell.get_basal_root_sections()[0]

# # get list of segments along one terminal path from this root
# dendrite_segs = get_first_dendrite(basal_root)


In [ ]:
# def get_dendrite_seg_data(dendrite_segs, sec_type=['all']):
#     '''subsets seg_data to only this dendrite's segments
#     sec_type gives list to look through'''
#     basal_segs, basal_seg_data = complex_cell.get_segments(sec_type)
#     dendrite_seg_data = [basal_seg_data[basal_segs.index(seg)] for seg in dendrite_segs]
#     return dendrite_seg_data

# dendrite_seg_data = get_dendrite_seg_data(dendrite_segs, sec_type=['dend'])

In [ ]:
# def calc_interspaced_synapses_positions_along_dendrite(dendrite_segs, dendrite_seg_data, synapse_spacing =20):
#     '''
#     synapse spacing determine distance between synapses
#     returns list of segments where synapses will go.
#      Could update to return sec and x instead for more precise placements, but segments should be isopotential so it should not matter '''
#     # Place synapses every 50 µm
#     segment_lengths = [seg.L for seg in dendrite_seg_data]
#     total_dendrite_length = sum(segment_lengths)
#     # Place synapses
#     synapse_spacing = 20  # in micrometers
#     current_length = 0
#     synapse_positions = []
#     for segment, seg_length in zip(dendrite_segs, segment_lengths):
#         start_pos = current_length
#         end_pos = current_length + seg_length
#         while start_pos <= end_pos:
#             if (start_pos // synapse_spacing) != (end_pos // synapse_spacing):
#                 # Calculate the exact position within the current segment
#                 relative_pos = ((synapse_spacing - (current_length % synapse_spacing)) / seg_length)
#                 # synapse_position = segment(relative_pos) segment not subrscriptable
#                 synapse_positions.append(segment)
#                 start_pos += synapse_spacing
#             else:
#                 break
#         current_length += seg_length
#     return synapse_positions

# def add_interspaced_synapses_along_dendrite(dendrite_segs, dendrite_seg_data, gmax_dist):
#     synapse_positions = calc_interspaced_synapses_positions_along_dendrite(dendrite_segs, dendrite_seg_data)
#     complex_cell.add_synapses_over_segments(
#         segments = None,
#         nsyn = None,
#         syn_mod = cell_builder.parameters.exc_syn_mod,
#         syn_params = cell_builder.parameters.exc_syn_params,
#         gmax = gmax_dist,
#         name = "exc",
#         density = False,
#         seg_probs = None,
#         release_p = None,
#         specific_segments=synapse_positions)
    
#     return synapse_positions
    

In [ ]:
# import numpy as np
# from functools import partial

# def binned_log_norm_dist(gmax_mean, gmax_std, gmax_scalar, size, clip):
# 	val = np.random.lognormal(gmax_mean, gmax_std, size)
# 	s = gmax_scalar * float(np.clip(val, clip[0], clip[1]))

# 	# Bin
# 	num_bins = 10
# 	bin_size = (clip[1] - clip[0]) / num_bins
# 	bins = np.arange(0, clip[1], bin_size)
# 	ind = np.digitize(s, bins)

# 	if ind == num_bins:
# 		return bins[-1]
# 	else:
# 		return bins[ind]

# gmax_exc_dist = partial(
# 	binned_log_norm_dist,#log_norm_dist, 
# 	cell_builder.parameters.exc_gmax_mean_0, 
# 	cell_builder.parameters.exc_gmax_std_0, 
# 	cell_builder.parameters.exc_scalar, 
# 	size = 1, 
# 	clip = cell_builder.parameters.exc_gmax_clip)

# synapse_positions = add_interspaced_synapses_along_dendrite(dendrite_segs, dendrite_seg_data, gmax_dist=gmax_exc_dist)

# complex_cell.add_synapses_over_segments(
# 	segments = None,
# 	nsyn = None,
# 	syn_mod = cell_builder.parameters.exc_syn_mod,
# 	syn_params = cell_builder.parameters.exc_syn_params,
# 	gmax = gmax_exc_dist,
# 	name = "exc",
# 	density = False,
# 	seg_probs = None,
# 	release_p = None,
# 	specific_segments=synapse_positions)

In [ ]:
# def assign_pc_for_each_synapse(cell, synapse_segments= None, synapses=None):
#     from presynaptic import PresynapticCell
#     pcs = []
#     segments, _ = cell.get_segments(['all'])
#     # if (synapses is None) and (synapse_segments is None):
#     #     raise(ValueError)
#     if synapses is None:
#         synapses = cell.synapses
#     elif synapse_segments is None:
#         synapse_segments = [synapse.hsyn.get_segment() for synapse in synapses]
#     for syn, syn_seg in zip(synapses, synapse_segments):
#         pc = PresynapticCell(segments.index(syn_seg))
#         pcs.append(pc)
#         syn.pc = pc
        
#     return pcs
    
# pcs = assign_pc_for_each_synapse(cell=complex_cell, synapse_segments=synapse_positions, synapses=None)

# from presynaptic import PresynapticCell
# pcs = []
# segments, _ = complex_cell.get_segments(['all'])
# for syn, syn_seg in zip(complex_cell.synapses, dendrite_segs):
#     pcs.append(PresynapticCell(segments.index(syn_seg)))

In [ ]:
# random_state = np.random.RandomState(cell_builder.parameters.numpy_random_state)
# np.random.seed(cell_builder.parameters.numpy_random_state)


# def assign_sequential_stim(random_state, pcs, interstim_interval = 50):
# 	'''first synapse will fire at {interstim_interval} ms, next synapse at 2*{interstim_interval} ms'''
# 	from spike_generator import PoissonTrainGenerator
# 	stim_time = interstim_interval
# 	for pc in pcs:
# 		firing_rates = np.zeros(cell_builder.parameters.h_tstop)
# 		firing_rates[stim_time] = 10000 # guarantee a spike?
# 		spike_train = PoissonTrainGenerator.generate_spike_train(
# 						lambdas = firing_rates, 
# 						random_state = random_state)
# 		pc.set_spike_train(spike_train.mean_fr, spike_train.spike_times)
# 		stim_time += interstim_interval
          

# random_state = np.random.RandomState(cell_builder.parameters.numpy_random_state)
# np.random.seed(cell_builder.parameters.numpy_random_state)

# assign_sequential_stim(random_state, pcs, interstim_interval = 50)

# interstim_interval = 50 # ms
# for pc in pcs:
# 	firing_rates = np.zeros(cell_builder.parameters.h_tstop)
# 	firing_rates[interstim_interval] = 10000
# 	spike_train = PoissonTrainGenerator.generate_spike_train(
# 					lambdas = firing_rates, 
# 					random_state = random_state)
# 	pc.set_spike_train(spike_train.mean_fr, spike_train.spike_times)
# 	interstim_interval += 50

In [ ]:
def get_first_dendrite(root_sec):
    # get list of segments along one terminal path from this root
    dendrite_segs = []
    sec_children = root_sec.children()
    while len(sec_children) > 0:
        for sec in sec_children:
            dendrite_segs.extend(list(sec))
        new_sec = sec_children[0]
        sec_children = new_sec.children()
    return dendrite_segs

def get_dendrite_seg_data(dendrite_segs, sec_type=['all']):
    '''subsets seg_data to only this dendrite's segments
    sec_type gives list to look through'''
    basal_segs, basal_seg_data = complex_cell.get_segments(sec_type)
    dendrite_seg_data = [basal_seg_data[basal_segs.index(seg)] for seg in dendrite_segs]
    return dendrite_seg_data

def calc_interspaced_synapses_positions_along_dendrite(dendrite_segs, dendrite_seg_data, synapse_spacing =20):
    '''
    synapse spacing determine distance between synapses
    returns list of segments where synapses will go.
     Could update to return sec and x instead for more precise placements, but segments should be isopotential so it should not matter '''
    # Place synapses every 50 µm
    segment_lengths = [seg.L for seg in dendrite_seg_data]
    total_dendrite_length = sum(segment_lengths)
    # Place synapses
    synapse_spacing = 20  # in micrometers
    current_length = 0
    synapse_positions = []
    for segment, seg_length in zip(dendrite_segs, segment_lengths):
        start_pos = current_length
        end_pos = current_length + seg_length
        while start_pos <= end_pos:
            if (start_pos // synapse_spacing) != (end_pos // synapse_spacing):
                # Calculate the exact position within the current segment
                relative_pos = ((synapse_spacing - (current_length % synapse_spacing)) / seg_length)
                # synapse_position = segment(relative_pos) segment not subrscriptable
                synapse_positions.append(segment)
                start_pos += synapse_spacing
            else:
                break
        current_length += seg_length
    return synapse_positions

def assign_pc_for_each_synapse(cell, synapse_segments= None, synapses=None):
    from presynaptic import PresynapticCell
    pcs = []
    segments, _ = cell.get_segments(['all'])
    # if (synapses is None) and (synapse_segments is None):
    #     raise(ValueError)
    if synapses is None:
        synapses = cell.synapses
    elif synapse_segments is None:
        synapse_segments = [synapse.hsyn.get_segment() for synapse in synapses]
    for syn, syn_seg in zip(synapses, synapse_segments):
        pc = PresynapticCell(segments.index(syn_seg))
        pcs.append(pc)
        syn.pc = pc
        
    return pcs

def assign_sequential_stim(random_state, pcs, interstim_interval = 50):
	'''first synapse will fire at {interstim_interval} ms, next synapse at 2*{interstim_interval} ms'''
	from spike_generator import PoissonTrainGenerator
	stim_time = interstim_interval
	for pc in pcs:
		firing_rates = np.zeros(cell_builder.parameters.h_tstop)
		firing_rates[stim_time] = 10000 # guarantee a spike?
		spike_train = PoissonTrainGenerator.generate_spike_train(
						lambdas = firing_rates, 
						random_state = random_state)
		pc.set_spike_train(spike_train.mean_fr, spike_train.spike_times)
		stim_time += interstim_interval

In [ ]:
import numpy as np
from functools import partial

def binned_log_norm_dist(gmax_mean, gmax_std, gmax_scalar, size, clip):
	val = np.random.lognormal(gmax_mean, gmax_std, size)
	s = gmax_scalar * float(np.clip(val, clip[0], clip[1]))

	# Bin
	num_bins = 10
	bin_size = (clip[1] - clip[0]) / num_bins
	bins = np.arange(0, clip[1], bin_size)
	ind = np.digitize(s, bins)

	if ind == num_bins:
		return bins[-1]
	else:
		return bins[ind]

def sequentially_stim_interspaced_synapses(spacing=50, interstim_interval=50):
    # dendrite path chosen is just the first basal root sec and the first of its children
    basal_root = complex_cell.get_basal_root_sections()[0]
    # get list of segments along one terminal path from this root
    dendrite_segs = get_first_dendrite(basal_root)

    dendrite_seg_data = get_dendrite_seg_data(dendrite_segs, sec_type=['dend'])

    gmax_exc_dist = partial(
	binned_log_norm_dist,#log_norm_dist, 
	cell_builder.parameters.exc_gmax_mean_0, 
	cell_builder.parameters.exc_gmax_std_0, 
	cell_builder.parameters.exc_scalar, 
	size = 1, 
	clip = cell_builder.parameters.exc_gmax_clip)

    #synapse_positions = add_interspaced_synapses_along_dendrite(dendrite_segs, dendrite_seg_data, gmax_dist=gmax_exc_dist, spacing = 50)
    synapse_positions = calc_interspaced_synapses_positions_along_dendrite(dendrite_segs, dendrite_seg_data, synapse_spacing =20)
    complex_cell.add_synapses_over_segments(
        segments = None,
        nsyn = None,
        syn_mod = cell_builder.parameters.exc_syn_mod,
        syn_params = cell_builder.parameters.exc_syn_params,
        gmax = gmax_exc_dist,
        name = "exc",
        density = False,
        seg_probs = None,
        release_p = None,
        specific_segments=synapse_positions)
    
    pcs = assign_pc_for_each_synapse(cell=complex_cell, synapse_segments=synapse_positions, synapses=None)

    random_state = np.random.RandomState(cell_builder.parameters.numpy_random_state)
    np.random.seed(cell_builder.parameters.numpy_random_state)

    assign_sequential_stim(random_state, pcs, interstim_interval = 50)
    

In [ ]:
# pick dendrite

# pick gmax

# pick synapse spacing and interstim interval

# use synapse spacing, dendrite_segs, gmax, to build synapses

# use interstim interval to sequentiall stim synapses

In [ ]:
def set_synapse_spike_trains(cell):
    for synapse in cell.synapsese:
        synapse.set_spike_train_from_pc()

set_synapse_spike_trains(complex_cell)

In [ ]:
cells = {
    'complex_cell': complex_cell
}
for cell_name, cell in cells.items():
    sim.set_all_recorders(cell, parameters)

In [ ]:
sim.simulate(complex_cell, parameters)

In [ ]:
import matplotlib.pyplot as plt
import analysis
cell_name = 'complex_cell'
id_to_use = 0
parameters.path = sim.path + 'try2'
use_nexus = False

print(cell_name, parameters.path)

v = analysis.DataReader.read_data(parameters.path, "v", parameters=parameters)

plt.plot(np.arange(0, len(v[id_to_use])) * parameters.h_dt, v[id_to_use], label=f"{cell_name} {'nexus' if use_nexus else 'soma'}")

This is interspaced, sequential stimulation.

Next will be same location n synapses with n interstim interval